In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pandas as pd

# get playlist id from Spotify
# pull up a playlist --> three dots -->  Share --> Copy Spotify URI
PlaylistExample = 'COPY SPOTIFY URI CODE'

# create spotipy client, client id and client secret come from https://developer.spotify.com/dashboard/applications/
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="INSERT CLIENT ID",
                                               client_secret="INSERT CLIENT SECRET ID"))

# create empty lists that will contain artist, song and album information
tracks = []
artist_blob = []
artist_name = []
song_name = []
album_name = []

# dataframe just used for visualization
combined = pd.DataFrame(columns = ['song', 'artist', 'album'])

# grabbing the entire song payload for the first song
result = sp.playlist_items(PlaylistExample, additional_types=['track'])
tracks.extend(result['items'])

# grabbing the entire song payload for each song in the playlist
while result['next']:
    result = sp.next(result)
    tracks.extend(result['items'])

# iterate through all items in the song payload
i = 0
for item in tracks:

    # grab artist information and append to list
    artist = item['track']['artists']
    artist_blob.append(artist)

    # grab album information and append to list
    album = item['track']['album']['name']
    album_name.append(album)

    # grab song information and append to list
    song = item['track']['name']
    song_name.append(song)

# need to specifically parse out artist names, since multiple can be present, and grabbing the number of artists depending on the length
for artist in artist_blob:

    # if the song has 1 artist, assign that artist's name
    if len(artist) == 1:
        artiste = artist[0]['name']
        artist_name.append(artiste)

    # if the song has 2 artists, append both together and assign it as the artist
    if len(artist) == 2:
        artiste = artist[0]['name'] + ' , ' + artist[1]['name']
        artist_name.append(artiste)

    # if the song has 3 artists, append the 3 together and assign it as the artist
    if len(artist) == 3:
        artiste = artist[0]['name'] + ' , ' + artist[1]['name'] + ' , ' + artist[2]['name']
        artist_name.append(artiste)

    # if the song has 4 or more artists, append the first 4 together and assign it as the artist
    if len(artist) >= 4:
        artiste = artist[0]['name'] + ' , ' + artist[1]['name'] + ' , ' + artist[2]['name'] + ' , ' + artist[3]['name']
        artist_name.append(artiste)

# this displays all the rows of the dataframe
pd.set_option('display.max_rows', None)

# appending data to dataframe for visualization, prints out all the songs in the playlist
combined['song'] = song_name
combined['artist'] = artist_name
combined['album'] = album_name
combined

In [ ]:
from youtubesearchpython import VideosSearch
import youtube_dl
import sys
import os
import  mutagen
from mutagen.flac import FLAC
from mutagen import MutagenError
import shutil
import time

# iterate through all songs, artists, and albums in the three lists
for (song, artist, album) in zip(song_name, artist_name, album_name):

    # search youtube for each song in the playlist
    videosSearch = VideosSearch(artist + ' ' + song + ' AUDIO', limit = 1, language = 'en')

    # append the search result to the list results
    results = []
    results.append(videosSearch.result())  

    # grabbing the top search result for a song
    video_link = results[0]['result'][0]['link']

    # defining the local download path and file name
    path = 'download_file_path/%(title)s.%(ext)s'

    # setting the parameters for youtube_dl
    # this one downloads the song as a FLAC file, and saves it to the path specified above
    params ={
        'outtmpl': path,
        'format': 'bestaudio/best',
        'extractaudio':True,
        'addmetadata':True,
        'postprocessors':[{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'flac',
            'preferredquality': '192',
        }],

    }

    # pass in the parameters above and download the song
    youtube = youtube_dl.YoutubeDL(params)
    youtube.download([video_link])

    # folder of downloaded song
    folder = r"download_file_path"
    
    # grabbing the song file
    audo_file = os.listdir(folder)[0]
    
    # full path of song file
    full = os.path.join(folder, audo_file )

    # specifying the song file is a FLAC file
    audio = FLAC(full) 

    # setting the artist, album and song metadata from the spotify information
    audio["ARTIST"] = artist
    audio["ALBUM"] = album
    audio["TITLE"] = song

    # Printing the metadata
    print(audio)

    # Saving the changes
    audio.save()

    # specifying the folder to move the song to
    target_dir = r'destination_file_path'

    # listing the songs new file name
    file_names = os.listdir(folder)
    print(file_names)

    # moving the song from the download folder to the new folder
    for file_name in file_names:
        shutil.move(os.path.join(folder, file_name), target_dir)

    # empty the list after each song
    results.clear()
    
    # delay 1 second between downloading each song
    time.sleep(1)